In [1]:
import pandas as pd
import numpy as np

In [2]:
file = 'D:\\学习课件\\405\\active_data.tsv'
df_raw = pd.read_csv(file, sep='\t')
print(df_raw.shape)
df_raw.head(1)

(5923, 61)


,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name Assigned by Curator or DataSource,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain.1,UniProt (SwissProt) Entry Name of Target Chain.1,UniProt (SwissProt) Primary ID of Target Chain.1,UniProt (SwissProt) Secondary ID(s) of Target Chain.1,UniProt (SwissProt) Alternative ID(s) of Target Chain.1,UniProt (TrEMBL) Submitted Name of Target Chain.1,UniProt (TrEMBL) Entry Name of Target Chain.1,UniProt (TrEMBL) Primary ID of Target Chain.1,UniProt (TrEMBL) Secondary ID(s) of Target Chain.1,UniProt (TrEMBL) Alternative ID(s) of Target Chain.1
0,50965826,CNC(=O)COc1cccc(Nc2ncc(F)c(Nc3ccc4OC(F)(F)C(=O...,InChI=1S/C21H17F3N6O4/c1-25-17(31)10-33-13-4-2...,MOXXQFNQDDSJHT-UHFFFAOYSA-N,50396071,CHEMBL2170582,Tyrosine-protein kinase SYK,Homo sapiens,1.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_raw.columns

Index(['BindingDB Reactant_set_id', 'Ligand SMILES', 'Ligand InChI',
       'Ligand InChI Key', 'BindingDB MonomerID', 'BindingDB Ligand Name',
       'Target Name Assigned by Curator or DataSource',
       'Target Source Organism According to Curator or DataSource', 'Ki (nM)',
       'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)', 'kon (M-1-s-1)', 'koff (s-1)',
       'pH', 'Temp (C)', 'Curation/DataSource', 'Article DOI', 'PMID',
       'PubChem AID', 'Patent Number', 'Authors', 'Institution',
       'Link to Ligand in BindingDB', 'Link to Target in BindingDB',
       'Link to Ligand-Target Pair in BindingDB', 'Ligand HET ID in PDB',
       'PDB ID(s) for Ligand-Target Complex', 'PubChem CID', 'PubChem SID',
       'ChEBI ID of Ligand', 'ChEMBL ID of Ligand', 'DrugBank ID of Ligand',
       'IUPHAR_GRAC ID of Ligand', 'KEGG ID of Ligand', 'ZINC ID of Ligand',
       'Number of Protein Chains in Target (>1 implies a multichain complex)',
       'BindingDB Target Chain  Sequence', 'PDB ID(s) of T

In [4]:
df = df_raw[['BindingDB MonomerID', 'Ligand SMILES', 'BindingDB Ligand Name', 'Ki (nM)', 'IC50 (nM)', 
            'Kd (nM)', 'EC50 (nM)', 'pH', 'Article DOI', 'PMID', 'PubChem AID', 'Patent Number']].copy(deep=True)
df.head(3)

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number
0,50396071,CNC(=O)COc1cccc(Nc2ncc(F)c(Nc3ccc4OC(F)(F)C(=O...,CHEMBL2170582,1.8,NaN,NaN,NaN,NaN,10.1021/jm201271b,22257213.0,NaN,NaN
1,50275934,Cc1cc(C)cc(Nc2nccc(n2)-c2ncc(CO)s2)c1,"(2-(2-(3,5-dimethylphenylamino)pyrimidin-4-yl)...",4,NaN,NaN,NaN,NaN,10.1016/j.bmcl.2008.09.106,18938080.0,NaN,NaN
2,50325991,COc1ccc(cc1OC)-c1cc2nccn2c(Nc2ncccc2C(N)=O)n1,"2-(7-(3,4-dimethoxyphenyl)imidazo[1,2-c]pyrimi...",7.5,NaN,NaN,NaN,NaN,10.1016/j.chembiol.2005.04.011,15975507.0,NaN,NaN


# 删除全部为空的无用列

In [5]:
print(df.shape)
df.dropna(axis=1, how='all', inplace=True)
print(df.shape)

(5923, 12)
(5923, 12)


# 查看四个活性值均缺失的行

In [6]:
df[df['Ki (nM)'].isnull() & df['IC50 (nM)'].isnull() & df['Kd (nM)'].isnull() & df['EC50 (nM)'].isnull()]

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number


或

In [16]:
df[df[['Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)']].isnull().sum(axis=1) == 4]

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number


# 查看包含活性值类型多于1种的行

In [69]:
df[df[['Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)']].notnull().sum(axis=1) > 1]

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number
2377,100350,CS(=O)(=O)N1CCN(CC1)c1ccc(Nc2ncc(C(N)=O)c(NC3C...,"US8501944, 372::US8501944, 375::US9868729, Exa...",NaN,15,4.5,NaN,NaN,NaN,NaN,NaN,US8501944


# 对活性值种类多于1的记录任取一种

In [19]:
df.loc[df[df[['Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)']].notnull().sum(axis=1) > 1].index.tolist(), 'IC50 (nM)'] = np.nan

In [21]:
df.loc[[2377]]

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number
2377,100350,CS(=O)(=O)N1CCN(CC1)c1ccc(Nc2ncc(C(N)=O)c(NC3C...,"US8501944, 372::US8501944, 375::US9868729, Exa...",NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,US8501944


# 将活性类别中的NaN替换为''

In [22]:
df.loc[:, ['Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)']] = df[['Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)']].fillna('')

# 将活性值合并到一列

In [23]:
df['activity'] = df['Ki (nM)'] + df['IC50 (nM)'] + df['Kd (nM)'] + df['EC50 (nM)']

In [25]:
df.head(1)

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number,activity
0,50396071,CNC(=O)COc1cccc(Nc2ncc(F)c(Nc3ccc4OC(F)(F)C(=O...,CHEMBL2170582,1.8,,,,NaN,10.1021/jm201271b,22257213.0,NaN,NaN,1.8


# 新建一列记录活性单位

In [26]:
for index, row in df.iterrows():
    if row['Ki (nM)'] != '':
        df.loc[index, 'unit'] = 'Ki (nM)'
    elif row['IC50 (nM)'] != '':
        df.loc[index, 'unit'] = 'IC50 (nM)'
    elif row['Kd (nM)'] != '':
        df.loc[index, 'unit'] = 'Kd (nM)'
    elif row['EC50 (nM)'] != '':
        df.loc[index, 'unit'] = 'EC50 (nM)'

In [28]:
df.head(1)

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number,activity,unit
0,50396071,CNC(=O)COc1cccc(Nc2ncc(F)c(Nc3ccc4OC(F)(F)C(=O...,CHEMBL2170582,1.8,,,,NaN,10.1021/jm201271b,22257213.0,NaN,NaN,1.8,Ki (nM)


# 查看不含“<”和“>”数据的统计量

In [156]:
df[~df['activity'].str.contains('<|>')]['activity'].astype('float64').describe()

count    5.192000e+03
mean     8.851735e+03
std      2.438747e+05
min      0.000000e+00
25%      1.140000e+01
50%      5.500000e+01
75%      3.691750e+02
max      1.200000e+07
Name: activity, dtype: float64

根据统计量得出的结论：<br />
1.最小值为0的数据，需要检查一下<br />
2.活性值的二分之一分位为55 nM，可以尝试以该值作为有活性和无活性的分界

# 检查活性为0的数据

In [79]:
df[~df['activity'].str.contains('<|>')][df[~df['activity'].str.contains('<|>')]['activity'].astype('float64') == 0]

,BindingDB MonomerID,Ligand SMILES,BindingDB Ligand Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),pH,Article DOI,PMID,PubChem AID,Patent Number,activity,unit
1540,140389,CCCn1cc(cn1)-c1cc(O[C@H](C)[C@H]2CNC(=O)C2)c2c...,"US8912173, 114::US9914735, Example 114",,0,,,7.5,NaN,NaN,NaN,US8912173,0,IC50 (nM)
1541,140390,CC(C)n1cc(cn1)-c1cc(O[C@H](C)[C@H]2CNC(=O)C2)c...,"US8912173, 115::US9914735, Example 115",,0,,,7.5,NaN,NaN,NaN,US8912173,0,IC50 (nM)
1542,140325,C[C@@H](Oc1cc(cc2ncccc12)-c1ccc(CN(C)C(C)=O)cc...,"US8912173, 51",,0,,,7.5,NaN,NaN,NaN,US8912173,0,IC50 (nM)


化合物来自于一篇专利，可能没有报道活性，直接删掉

# 获取活性值为0数据的索引

In [82]:
index_del = df[~df['activity'].str.contains('<|>')][df[~df['activity'].str.contains('<|>')]['activity'].astype('float64') == 0].index.to_list()
index_del

[1540, 1541, 1542]

# 删除活性值为0的数据

In [84]:
print(df.shape)
df.drop(index=index_del, inplace=True)
print(df.shape)

(5923, 14)
(5920, 14)


# 对不含“<”和“>”的数据取对数，并做直方图

In [94]:
df['logA'] = df[~df['activity'].str.contains('<|>')]['activity'].astype('float64').apply(np.log)
df['logA'].describe()

count    5189.000000
mean        4.103849
std         2.473116
min        -4.605170
25%         2.442347
50%         4.007333
75%         5.912692
max        16.300417
Name: logA, dtype: float64

# 处理重复数据

In [191]:
# 重复数据数量
df.shape[0] - len(df.groupby('Ligand SMILES'))

967

In [188]:
for smi, group in df.groupby('Ligand SMILES'):
    if len(group) >1:
        

CC(C)c1ccnc(Nc2cc(C)cc(c2)-c2cnc(s2)C(C)(O)C2CCC(CC2)C(O)=O)n1 1186    212239
3744    212238
Name: BindingDB MonomerID, dtype: int64
CCOC(=O)C1CCC(O)(CC1)c1ccc(cn1)-c1cc(C)cc(Nc2cc(ccn2)C(F)(F)F)n1 3082    182043
3115    182044
Name: BindingDB MonomerID, dtype: int64
Cc1cc(Nc2cc(ccn2)C(F)(F)F)nc(c1)-c1ccc(NC2CCC(CC2)C(O)=O)nc1 1813    182090
5401    182089
Name: BindingDB MonomerID, dtype: int64
Cc1cc(Nc2cc(ccn2)C(F)(F)F)nc(c1)-c1ccc(nc1)C1(O)CCC(CC1)C(O)=O 1332    182051
5409    182050
Name: BindingDB MonomerID, dtype: int64
Cc1cc(Nc2nccc(C)n2)cc(c1)-c1cnc(s1)C(C)(O)C1CCC(CC1)C(O)=O 2282    212234
5388    212233
Name: BindingDB MonomerID, dtype: int64
